In [7]:
pdf_file_path = 'C:/Users/yrui7/Documents/vscode/Arxiv/AI/ai_1.pdf'
question_text="what's Conclusion?"

In [8]:
# read text from pdf files 

import PyPDF2

# Path to your PDF file

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text().lower()
    return text

# Extract text from the PDF
pdf_text = extract_text_from_pdf(pdf_file_path)

# Print the text extracted from the PDF
print(pdf_text)

 
 
 
* equal contribution  raise  – radiology ai  safe ty, an end-
to-end lifecycle approach   
 
m. jorge cardoso  * 
school of biomedical engineering and imaging sciences, king’s college london, london, uk  
 
julia moosbauer  * 
deepc gmbh, munich, germany ;  
department of statistics, ludwig maximilian university munich, munich, germany  
 
tessa s. cook  
department of radiology, perelman school of medicine at the university of pennsylvania, 
philadelphia, pa, usa  
 
b. selnur erdal,  
center  for augmented intelligence in imaging, department of radiology, mayo clinic, jacksonville, 
florida, usa  
 
brad genereaux  
nvidia, santa clara, ca, usa  
 
vikash gupta  
center for augmented intelligence in imaging,  department of radiology,  mayo clinic, jacksonville, 
florida, usa  
 
bennett a. landman  
vanderbilt university, nashville, tennessee, usa  
 
tiarna lee  
school of biomedical engineering and imaging sciences, king’s college london, london, uk  
 
parashkev nachev  
ucl

In [9]:
import os
import re
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

question_embedding = client.embeddings.create(input=question_text, model="text-embedding-ada-002").data[0].embedding 
question_embedding

# Split the document into paragraphs based on double newline characters
paragraphs = re.split(r'\n\s*\n', pdf_text)

# Group paragraphs into sets of ten for chunking
chunks = [paragraphs[i:i+10] for i in range(0, len(paragraphs), 10)]

# Initialize an empty list to store the embeddings
embeddings = []

# Initialize variables to store maximum similarity and corresponding chunk index
max_similarity = -1  # Initialize with a value lower than possible cosine similarity (-1 to 1)
max_similarity_chunk_index = -1

# Loop through each chunk and obtain embeddings using OpenAI's API
for idx, chunk in enumerate(chunks, start=1):
    chunk_text = '\n\n'.join(chunk)
    response = client.embeddings.create(input=chunk_text, model="text-embedding-ada-002")
    embeddings.append(response.data[0].embedding)
    chunk_embedding = response.data[0].embedding
    print(f"Embedding for Chunk {idx}: {response.data[0].embedding}\n")
    similarity = np.dot(question_embedding, chunk_embedding) / (np.linalg.norm(question_embedding) * np.linalg.norm(chunk_embedding))
    if similarity > max_similarity:
        max_similarity = similarity
        max_similarity_chunk_index = idx
        max_similarity_chunk_content = chunk
    print(f"Cosine similarity with Chunk {idx}: {similarity}\n")
    
print(f"Chunk with the highest similarity (Chunk {max_similarity_chunk_index}): {max_similarity}")


Embedding for Chunk 1: [-0.012303822673857212, -0.00554001796990633, 0.002649095840752125, -0.022746317088603973, 0.0016653029015287757, 0.01956593617796898, -0.0031712204217910767, -0.014406977221369743, -0.028813954442739487, -0.0090281767770648, 0.010574398562312126, 0.008647117763757706, 0.01177620142698288, 0.012450383976101875, 0.011189956218004227, 0.02144191972911358, 0.02490076795220375, -0.0022185719572007656, -0.011930091306567192, -0.008822991512715816, -0.012853427790105343, 0.02564823068678379, -0.021749699488282204, 0.008537196554243565, -0.01632693037390709, 0.014062558300793171, 0.033064231276512146, -0.02539907582104206, -0.01233313512057066, -0.0030100031290203333, 0.01591655984520912, 0.0073537142015993595, -0.024490395560860634, -0.024490395560860634, -0.0031510682310909033, -0.0056536030024290085, -0.006467018276453018, 0.006591595243662596, 0.044789139181375504, -0.013051285408437252, 0.015330313704907894, 0.01090416219085455, 0.00616656756028533, -0.024812830612

In [10]:
# Output the content of the chunk with the highest similarity
print("Content of the chunk with the highest similarity:")
print('\n\n'.join(max_similarity_chunk_content))
Chunk_with_the_highest_similarity='\n\n'.join(max_similarity_chunk_content)

Content of the chunk with the highest similarity:
figure 4 - depiction of different types of system level safety mechanisms  
the platform should define operating thresholds for these checks based on model specifics and 
risk tolerance. detected issues should trigger alerts to the underlying ai medical device manufacturer,  
along with deferring the results from getting to clinical users. this allows for proactive debugging and 
re-training of models if needed.  

3.3. deferral of algorithmic decisions  
with the introduction of platform -level input and output guardrail mechanisms, it is important to 
ensure these acts  as a fail -safe. if an algorithmic output is deemed not to be of sufficient quality 
according to a locally calibrated threshold, it should be deferred to the most appropriate downstream 
expert clinician while providing a transparent explanation for the def erral reason. it is important to 
note that while deferral mechanisms  do not fully mitigate but minimis e risk 

In [11]:
response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Answer the question based on the following: \n{Chunk_with_the_highest_similarity} \n Question:{question_text} "}
        ],
        temperature=0.7,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        max_tokens=200,
        stop=None
    )
response.choices[0].message.content

"The conclusion is that the AI systems should have well-defined operating thresholds, guardrails, and deferral mechanisms in place to ensure system level safety. Model outputs that fail these checks should be deferred to human experts for review. These deferral mechanisms should be integrated with the patient's electronic health record or the radiological information system. Model execution should be consistently monitored, with key performance indicators tracked for auditing and management purposes. During the post-market surveillance phase, performance drift should be monitored to identify any potential degradation. If degradation is observed, a full clinical review and quality control study redo will be necessary. These steps will ensure comprehensive visibility into AI systems and uphold the highest level of patient care."